In [2]:
def extract_and_print_exif(image_path):
    """
    提取图片的 EXIF 信息并按照指定规则打印，忽略 thumbnail 信息。
    
    :param image_path: 图片文件路径
    :return: 原始 EXIF 数据（从 piexif.load 直接返回）
    """
    try:
        # 加载 EXIF 数据
        exif_data = piexif.load(image_path)
        
        for ifd_name, ifd_content in exif_data.items():
            # 跳过缩略图
            if ifd_name == "thumbnail":
                continue

            print(f"--- {ifd_name} ---")
            
            if isinstance(ifd_content, dict):
                for tag, value in ifd_content.items():
                    # 获取标签名称
                    tag_name = piexif.TAGS[ifd_name].get(tag, {"name": f"Unknown tag ({tag})"}).get("name")
                    
                    # 判断字段类型
                    if isinstance(value, bytes):
                        # 保留 ComponentsConfiguration 和 MakerNote 的原始输出
                        if tag_name in ["ComponentsConfiguration", "MakerNote"]:
                            print(f"{tag_name}: {value} (raw bytes)")
                        else:
                            # 其他 bytes 尝试解码为字符串
                            try:
                                value_str = value.decode("utf-8", errors="ignore")
                                print(f"{tag_name}: {value_str} (decoded from bytes)")
                            except UnicodeDecodeError:
                                print(f"{tag_name}: {value} (raw bytes, decode failed)")
                    else:
                        # 非 bytes 类型直接打印
                        print(f"{tag_name}: {value}")
            else:
                print(f"Non-dict content in {ifd_name}: {ifd_content}")
        
        return exif_data  # 返回原始 EXIF 数据

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [3]:
import shutil

def write_exif_to_image(exif_data, image_path):
    """
    将 EXIF 数据写入另一张图片中，但不包含 thumbnail 信息。
    
    :param exif_data: 原始 EXIF 数据（从 piexif.load 返回的结果）
    :param source_image_path: 原始图片路径，用于读取图像数据
    :param target_image_path: 目标图片路径，写入新 EXIF 数据后保存
    """
    try:

        # 获取文件夹路径和原始文件名
        folder_path = os.path.dirname(image_path)
        file_name = os.path.basename(image_path)
        
        # 创建备份文件夹 "bak"，如果不存在
        bakeup_folder_path = os.path.join(folder_path, "bak")
        if not os.path.exists(bakeup_folder_path):
            os.makedirs(bakeup_folder_path)
        backup_path = os.path.join(bakeup_folder_path, file_name)

        # 保存原图片为备份
        shutil.copy(image_path, backup_path)
        print(f"原图片已备份为：{backup_path}")
        
        # 移除 thumbnail 信息
        if "thumbnail" in exif_data:
            exif_data.pop("thumbnail")

        # 提取文件名中的日期部分（假设日期部分格式为 20241019234238）
        base_name = file_name.split('.')[0]
        date_str = base_name.split('_')[-1]  # 获取日期部分（例如：20241019234238）
        
        # 格式化为 EXIF 所需的日期时间格式 YYYY:MM:DD HH:MM:SS
        date_time_str = f"{date_str[:4]}:{date_str[4:6]}:{date_str[6:8]} {date_str[8:10]}:{date_str[10:12]}:{date_str[12:14]}"
        
        # 将日期时间字符串转换为字节流
        date_time_bytes = date_time_str.encode()
        
        # 修改 EXIF 中的 DateTimeDigitized、DateTimeOriginal 和 DateTime 字段
        if "0th" not in exif_data:
            exif_data["0th"] = {}
        exif_data["0th"][piexif.ImageIFD.DateTime] = date_time_bytes  # 修改 DateTime
        
        if "Exif" not in exif_data:
            exif_data["Exif"] = {}
        exif_data["Exif"][piexif.ExifIFD.DateTimeOriginal] = date_time_bytes  # 修改 DateTimeOriginal
        exif_data["Exif"][piexif.ExifIFD.DateTimeDigitized] = date_time_bytes  # 修改 DateTimeDigitized
        

        # 获取文件名，并将其写入 ImageDescription 字段
        if "0th" not in exif_data:
            exif_data["0th"] = {}
        exif_data["0th"][piexif.ImageIFD.ImageDescription] = file_name.split('.')[0].encode()  # 写入 ImageDescription 字段
    
        # 将 EXIF 数据转换为二进制格式
        exif_bytes = piexif.dump(exif_data)
        
        # 读取原始图片数据
        with open(image_path, "rb") as image_file:
            image_data = image_file.read()
        
        # 将 EXIF 数据写入到目标图片
        with open(image_path, "wb") as image_file:
            piexif.insert(exif_bytes, image_data, image_path)
        
        print(f"{image_path} 写入 EXIF 成功")
    except Exception as e:
        print(f"{image_path} 写入 EXIF 失败: {e}")

In [4]:
import os
import imghdr
import piexif

def process_images_in_folder(folder_path):
    """
    遍历文件夹中的所有 .jpg 文件，并对每个文件执行 write_exif_to_image_without_thumbnail 函数。

    :param folder_path: 目标文件夹路径
    """
    try:
        # 遍历文件夹中的所有文件
        for file_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, file_name)
            # 确保只处理 .jpg 文件
            if file_name.lower().endswith('.jpg') and imghdr.what(image_path) == 'jpeg':
                
                # 加载图片的 EXIF 数据
                exif_data = piexif.load(image_path)
                
                # 判断 EXIF 是否包含 Make 或 Model 信息，如果包含则跳过
                if '0th' in exif_data:
                    # 获取 Make 和 Model 的值，如果存在则跳过
                    make = exif_data['0th'].get(piexif.ImageIFD.Make)
                    model = exif_data['0th'].get(piexif.ImageIFD.Model)

                    if make or model:  # 如果包含 Make 或 Model 信息
                        print(f"跳过文件 {file_name}，因为它包含 Make 或 Model 信息")
                        continue  # 跳过当前图片文件，继续下一个文件
                else:
                    print(f"跳过文件 {file_name}，因为它包含 Make 或 Model 信息")
                    continue  # 跳过当前图片文件，继续下一个文件
                
                # 调用 write_exif_to_image_without_thumbnail 函数处理 EXIF 数据
                write_exif_to_image(exif_raw, image_path)
                print(f"处理完成: {file_name}")
    
    except Exception as e:
        print(f"操作文件夹时出错: {e}")

In [5]:
# 执行调用
exif_raw = extract_and_print_exif("data/ps5_exif_source.jpg")
folder_path = "/Volumes/Apple/PS5/CREATE/Screenshots/其他"  # 替换为你的文件夹路径
process_images_in_folder(folder_path)

--- 0th ---
ImageDescription: 黑神话：悟空_20240915023312 (decoded from bytes)
Make: Sony Interactive Entertainment Inc. (decoded from bytes)
Model: PlayStation(R)5 (decoded from bytes)
Orientation: 1
XResolution: (72, 1)
YResolution: (72, 1)
ResolutionUnit: 2
Software: 24.06-10.00.00.46-00.00.00.0.1 (decoded from bytes)
DateTime: 2024:09:15 02:33:12 (decoded from bytes)
YCbCrPositioning: 1
ExifTag: 300
--- Exif ---
ExifVersion: 0221 (decoded from bytes)
DateTimeOriginal: 2024:09:15 02:33:12 (decoded from bytes)
DateTimeDigitized: 2024:09:15 02:33:12 (decoded from bytes)
ComponentsConfiguration: b'\x01\x02\x03\x00' (raw bytes)
MakerNote: b'SONY PS\x00\x00\x00\x00\x00SCE\x01\x00\x00\x00\x00\x00\xc6\x00\x06TXXX\x00\x00\x00\r\x00\x00\x03XXXPEFV\x001.0\x00TXXX\x00\x00\x00.\x00\x00\x03XXXGCID\x00HP6545-PPSA23226_00-GAME000000000000\x00TXXX\x00\x00\x00\x13\x00\x00\x03XXXGPNM\x00PPSA23226\x00TXXX\x00\x00\x00\x1c\x00\x00\x03XXXALBM\x00\xe9\xbb\x91\xe7\xa5\x9e\xe8\xaf\x9d\xef\xbc\x9a\xe6\x82\x9f\xe7\

In [6]:
# 执行调用
folder_path = "/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空" # 替换为你的文件夹路径
process_images_in_folder(folder_path)

原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/bak/黑神话：悟空_20241018225954.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/黑神话：悟空_20241018225954.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241018225954.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/bak/黑神话：悟空_20241015003605.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/黑神话：悟空_20241015003605.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241015003605.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/bak/黑神话：悟空_20241013164640.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/黑神话：悟空_20241013164640.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164640.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/bak/黑神话：悟空_20241013164638.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/黑神话：悟空_20241013164638.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164638.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/bak/黑神话：悟空_20241013164559.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/黑神话：悟空/黑神话：悟空_20241013164559.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164559.jpg
原图片已备份为：/Volume

In [7]:
# 执行调用
folder_path = "/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南" # 替换为你的文件夹路径
process_images_in_folder(folder_path)

原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/bak/宇宙机器人无线控制器使用指南_20241020231405.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/宇宙机器人无线控制器使用指南_20241020231405.jpg 写入 EXIF 成功
处理完成: 宇宙机器人无线控制器使用指南_20241020231405.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/bak/宇宙机器人无线控制器使用指南_20241020222503.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/宇宙机器人无线控制器使用指南_20241020222503.jpg 写入 EXIF 成功
处理完成: 宇宙机器人无线控制器使用指南_20241020222503.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/bak/宇宙机器人无线控制器使用指南_20241020222500.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/宇宙机器人无线控制器使用指南_20241020222500.jpg 写入 EXIF 成功
处理完成: 宇宙机器人无线控制器使用指南_20241020222500.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/bak/宇宙机器人无线控制器使用指南_20241020220816.jpg
/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无线控制器使用指南/宇宙机器人无线控制器使用指南_20241020220816.jpg 写入 EXIF 成功
处理完成: 宇宙机器人无线控制器使用指南_20241020220816.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Screenshots/宇宙机器人无

In [8]:
# 执行调用
folder_path = "/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空" # 替换为你的文件夹路径
process_images_in_folder(folder_path)

原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/bak/黑神话：悟空_20241018223037.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/黑神话：悟空_20241018223037.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241018223037.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/bak/黑神话：悟空_20241013164747.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/黑神话：悟空_20241013164747.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164747.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/bak/黑神话：悟空_20241013164714.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/黑神话：悟空_20241013164714.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164714.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/bak/黑神话：悟空_20241013164639.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/黑神话：悟空_20241013164639.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013164639.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/bak/黑神话：悟空_20241013010412.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/黑神话：悟空/黑神话：悟空_20241013010412.jpg 写入 EXIF 成功
处理完成: 黑神话：悟空_20241013010412.jpg
原图片已备份为：/Volume

In [10]:
# 执行调用
folder_path = "/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南" # 替换为你的文件夹路径
process_images_in_folder(folder_path)

原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/bak/宇宙机器人无线控制器使用指南_20241020231451.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/宇宙机器人无线控制器使用指南_20241020231451.jpg 写入 EXIF 成功
处理完成: 宇宙机器人无线控制器使用指南_20241020231451.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/bak/TOTAL TIME_2024102023140700.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/TOTAL TIME_2024102023140700.jpg 写入 EXIF 成功
处理完成: TOTAL TIME_2024102023140700.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/bak/FROZEN RUN_2024102023140600.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/FROZEN RUN_2024102023140600.jpg 写入 EXIF 成功
处理完成: FROZEN RUN_2024102023140600.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/bak/JETPACK RUN_2024102023115700.jpg
/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/JETPACK RUN_2024102023115700.jpg 写入 EXIF 成功
处理完成: JETPACK RUN_2024102023115700.jpg
原图片已备份为：/Volumes/Apple/PS5/CREATE/Video Clips/宇宙机器人无线控制器使用指南/bak/TO